In [6]:
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt

In [38]:
def nearestNeighbors(s0, S, n):
    orderedNeighbors = np.argsort(la.norm(s0 - S[:-1],axis=1))
    return orderedNeighbors[1:n+1]

In [60]:
# Step 1
N = 5
r = 4

"""
X = np.ones((N, r))
for i in range(N):
    for j in range(r):
        X[i,j] = i + j
"""
"""X = np.array([[0,0,1,1],
              [0,0,1,1],
              [-0.5,0,1,1],
              [0,0,1.2,1],
              [0,0,1,1]])
"""
X = np.array([[0,0,0,0],
              [0,0,0,0.5],
              [1,1,1,1],
              [1,1,1.2,1],
              [0.9,1,1,1]])
print(X)

[[0.  0.  0.  0. ]
 [0.  0.  0.  0.5]
 [1.  1.  1.  1. ]
 [1.  1.  1.2 1. ]
 [0.9 1.  1.  1. ]]


In [61]:
# Step 2: chose parameters(how ??)
a = 1    # bias parameter
k = 1  # 
s = 0 # delay

In [62]:
# Step 3: Weighted Embedding

print("Embedding Dimensions: ",N-s, r*(s+1))
Xemb = np.zeros((N-s, r*(s+1)))
for e in range(s+1):
    Xemb[:,e*r:(e+1)*r] = np.exp(-k*e) * X[s-e:N-e,:]
    
print(Xemb, "Works as expected")

Embedding Dimensions:  5 4
[[0.  0.  0.  0. ]
 [0.  0.  0.  0.5]
 [1.  1.  1.  1. ]
 [1.  1.  1.2 1. ]
 [0.9 1.  1.  1. ]] Works as expected


In [63]:
# Step 4: Generate Gaussian Random variables
m = Xemb.shape[1] # TODO: find out where m comes from

pi = np.random.normal(0,1,(m, r*(s+1)))

In [64]:
# Step 5: orthonormalize w/ QR

Q, R = la.qr(pi)
pihat = Q # is this a random orthonormal basis?

np.set_printoptions(suppress=True)
# print(pi, pihat, np.sum(pi))

In [65]:
# Step 6: form compressed states
compressedStates = np.zeros(Xemb.shape) # corresponds to y_i hat

for i in range(Xemb.shape[0]):
    # print(pihat.shape, Xemb[i,:,None].shape)
    compressedStates[i,:] = (pihat @ Xemb[i,:,None]).T

In [66]:
# Step 7: find those neighbors

neighborIndices = np.zeros((Xemb.shape[0],k))

for i in range(Xemb.shape[0]):
    neighborIndices[i,:] = np.argsort(la.norm(Xemb[i] - Xemb[:-1],axis=1))[1:k+1]
    
print(neighborIndices)

[[1.]
 [0.]
 [3.]
 [2.]
 [3.]]


In [67]:
# Step 8: 

neighborMatrix = np.zeros((Xemb.shape[0],Xemb.shape[0]))
for point in range(neighborIndices.shape[0]):
    for neighborIndex in range(neighborIndices.shape[1]):
        neighborMatrix

SyntaxError: unexpected EOF while parsing (<ipython-input-67-ec0868b4468b>, line 5)